In [1]:
# Import libraries
import locale

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder 

from datetime import datetime
from locale import atof

In [2]:
# Define function that transforms date into value corresponding to number of week in year 
# that it belongs to.
def day_to_week_of_year(date_to_transform):
    return datetime.strptime(date_to_transform + "/2017", '%d/%B/%Y').isocalendar()[1]

In [3]:
# Load data
data = pd.read_csv('train.csv', sep=';', decimal=',')

In [4]:
# See data column names.
data.columns

Index(['ID', 'TIMESTAMP', 'WEBSITE', 'GDS', 'DEPARTURE', 'ARRIVAL', 'ADULTS',
       'CHILDREN', 'INFANTS', 'TRAIN', 'HAUL_TYPE', 'DISTANCE', 'DEVICE',
       'TRIP_TYPE', 'PRODUCT', 'SMS', 'EXTRA_BAGGAGE', 'NO_GDS'],
      dtype='object')

In [5]:
# See how many missing values the dataset has.
print(data.isnull().sum())

# Since there are missing values only in variable 'DEVICE' and they are very few we'll 
# just remove them. If there were much more we could use imputation methods to fill this
# missing values.
data = data[pd.notnull(data['DEVICE'])]
data.reset_index(drop=True)

print(data.isnull().sum())

ID                 0
TIMESTAMP          0
WEBSITE            0
GDS                0
DEPARTURE          0
ARRIVAL            0
ADULTS             0
CHILDREN           0
INFANTS            0
TRAIN              0
HAUL_TYPE          0
DISTANCE           0
DEVICE           133
TRIP_TYPE          0
PRODUCT            0
SMS                0
EXTRA_BAGGAGE      0
NO_GDS             0
dtype: int64
ID               0
TIMESTAMP        0
WEBSITE          0
GDS              0
DEPARTURE        0
ARRIVAL          0
ADULTS           0
CHILDREN         0
INFANTS          0
TRAIN            0
HAUL_TYPE        0
DISTANCE         0
DEVICE           0
TRIP_TYPE        0
PRODUCT          0
SMS              0
EXTRA_BAGGAGE    0
NO_GDS           0
dtype: int64


In [6]:
# See distribution of classes in target variable.
data['EXTRA_BAGGAGE'].value_counts()

# Classes are imbalanced. There are much more samples belonging to the 'False' class than 
# to the 'True' class.

False    40068
True      9799
Name: EXTRA_BAGGAGE, dtype: int64

In [7]:
# Review date variables: 'TIMESTAMP', 'DEPARTURE' and 'ARRIVAL'
print(data['TIMESTAMP'][0:5])
print(data['DEPARTURE'][0:5])
print(data['ARRIVAL'][0:5])

# All three date variables have the same format 'day_number/month_name'.
# We'll transform these values to numeric values corresponding to number 
# of week in year that the date belongs to.

data['TIMESTAMP'] = data['TIMESTAMP'].apply(day_to_week_of_year)
data['DEPARTURE'] = data['DEPARTURE'].apply(day_to_week_of_year)
data['ARRIVAL'] = data['ARRIVAL'].apply(day_to_week_of_year)

print(data['TIMESTAMP'][0:5])
print(data['DEPARTURE'][0:5])
print(data['ARRIVAL'][0:5])

0    01/July
1    01/July
2    01/July
3    01/July
4    01/July
Name: TIMESTAMP, dtype: object
0      22/July
1      29/July
2      29/July
3      24/July
4    11/August
Name: DEPARTURE, dtype: object
0      25/July
1      29/July
2    19/August
3    04/August
4    11/August
Name: ARRIVAL, dtype: object
0    26
1    26
2    26
3    26
4    26
Name: TIMESTAMP, dtype: int64
0    29
1    30
2    30
3    30
4    32
Name: DEPARTURE, dtype: int64
0    30
1    30
2    33
3    31
4    32
Name: ARRIVAL, dtype: int64


In [8]:
# Transform string variables 'WEBSITE', 'DEVICE', 'HAUL_TYPE', 'TRIP_TYPE' and 'PRODUCT' 
# to categorical using LabelEncoder.
# IMPORTANT: form 'WEBSITE' variable, we are not sure, all levels that this categorical 
# variable can take, arein the train.csv dataset.
label_encoder = LabelEncoder()

print(data['WEBSITE'].dtype)
data['WEBSITE'] = pd.Series(label_encoder.fit_transform(data['WEBSITE'])).values
print(data['WEBSITE'][0:5])

print(data['DEVICE'].dtype)
data['DEVICE'] = pd.Series(label_encoder.fit_transform(data['DEVICE'])).values
print(data['DEVICE'][0:5])

print(data['HAUL_TYPE'].dtype)
data['HAUL_TYPE'] = pd.Series(label_encoder.fit_transform(data['HAUL_TYPE'])).values
print(data['HAUL_TYPE'][0:5])

print(data['TRIP_TYPE'].dtype)
data['TRIP_TYPE'] = pd.Series(label_encoder.fit_transform(data['TRIP_TYPE'])).values
print(data['TRIP_TYPE'][0:5])

print(data['PRODUCT'].dtype)
data['PRODUCT'] = pd.Series(label_encoder.fit_transform(data['PRODUCT'])).values
print(data['PRODUCT'][0:5])

object
0    11
1    18
2    54
3    50
4    11
Name: WEBSITE, dtype: int64
object
0    3
1    2
2    3
3    2
4    0
Name: DEVICE, dtype: int64
object
0    1
1    0
2    0
3    1
4    0
Name: HAUL_TYPE, dtype: int64
object
0    2
1    1
2    2
3    0
4    1
Name: TRIP_TYPE, dtype: int64
object
0    1
1    1
2    1
3    1
4    1
Name: PRODUCT, dtype: int64


In [9]:
# ['ID', 'TIMESTAMP', 'WEBSITE', 'GDS', 'DEPARTURE', 'ARRIVAL', 'ADULTS',
#  'CHILDREN', 'INFANTS', 'TRAIN', 'HAUL_TYPE', 'DISTANCE', 'DEVICE',
#  'TRIP_TYPE', 'PRODUCT', 'SMS', 'EXTRA_BAGGAGE', 'NO_GDS']

# Check all integer variables 'GDS', 'NO_GDS', 'ADULTS', 'CHILDREN' and 'INFANTS', have 
# their corresponding dtype in the pandas DataFrame.
print(data['GDS'].dtype)
print(data['NO_GDS'].dtype)
print(data['ADULTS'].dtype)
print(data['CHILDREN'].dtype)
print(data['INFANTS'].dtype)

int64
int64
int64
int64
int64


In [10]:
# Check all boolean variables 'TRAIN', 'SMS' and 'EXTRA_BAGGAGE' have their corresponding 
# dtype in the pandas DataFrame.
print(data['TRAIN'].dtype)
print(data['SMS'].dtype)
print(data['EXTRA_BAGGAGE'].dtype)

bool
bool
bool


In [11]:
# Check float variable 'DISTANCE' has it's corresponding dtype in the pandas DataFrame.
print(data['DISTANCE'].dtype)

float64


In [12]:
# Drop 'ID' variable since it is useless.
data = data.drop('ID', 1)
data.reset_index(drop=True)

,TIMESTAMP,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,EXTRA_BAGGAGE,NO_GDS
0,26,11,1,29,30,1,0,0,False,1,628.844,3,2,1,True,False,0
1,26,18,0,30,30,1,0,0,False,0,1281.430,2,1,1,False,False,1
2,26,54,2,30,33,1,0,0,False,0,1730.350,3,2,1,True,False,0
3,26,50,0,30,31,1,0,0,False,1,652.702,2,0,1,False,False,2
4,26,11,0,32,32,1,0,0,False,0,1717.850,0,1,1,False,False,1
5,26,12,1,27,29,1,0,0,False,0,1065.560,0,0,1,False,False,1
6,26,11,1,32,32,2,0,0,False,0,959.105,0,1,1,True,False,0
7,26,11,1,29,30,2,0,0,False,0,1469.850,0,2,1,False,False,0
8,26,42,1,33,33,1,0,0,False,0,600.931,0,1,1,True,True,0
9,26,54,0,33,34,1,0,0,False,0,3024.020,2,2,1,False,False,2


In [13]:
# Store 'clean' dataset into new .csv file
data.to_csv('clean_train.csv', index=False, sep=';')